In [25]:
###CONFIGURAÇÕES INICIAIS###

# Instalar as bibliotecas necessárias

import os
import pandas as pd
import holidays
from datetime import datetime, timedelta

# Caminho para salvar o arquivo no Google Drive
file_path = r'C:\Users\yurib\pjf-engenharia-de-dados\algo'

In [26]:


### CRIA DIM_DATA ###


# Gerar uma lista de datas de 2015 até hoje + 1 dia
end_date = datetime.now() + timedelta(days=1)
dates = pd.date_range(start='2015-01-01', end=end_date)

# Obter feriados no Brasil
br_holidays = holidays.Brazil(years=range(2015, end_date.year + 1))

# Criar um DataFrame com as datas
df = pd.DataFrame({'dt_data': dates})

# Adicionar uma coluna indicando se a data é útil (não é feriado e não é fim de semana)
df['dt_util'] = df['dt_data'].apply(lambda x: x not in br_holidays and x.weekday() < 5)

# Adicionar colunas de dia da semana, mês/ano, trimestre e ano
df['dt_diasemana'] = df['dt_data'].dt.strftime('%A')
df['dt_diasemana_ord'] = df['dt_data'].dt.weekday
df['dt_mesano'] = df['dt_data'].dt.strftime('%B-%Y')
df['dt_mesano_ord'] = df['dt_data'].dt.month + (df['dt_data'].dt.year - 2015) * 12
df['dt_trimestre'] = df['dt_data'].dt.to_period('Q').astype(str).str.replace('Q', '-Q')
df['dt_trimestre_ord'] = df['dt_data'].dt.quarter + (df['dt_data'].dt.year - 2015) * 4
df['dt_ano'] = df['dt_data'].dt.year

# Reorganizar as colunas conforme o formato especificado
df = df[['dt_util', 'dt_data', 'dt_diasemana', 'dt_diasemana_ord',
         'dt_mesano', 'dt_mesano_ord', 'dt_trimestre', 'dt_trimestre_ord', 'dt_ano']]

# Exibir as primeiras linhas do DataFrame
# print(df.head())


In [27]:

### CRIA PEDIDOS ###

pedidos = pd.read_csv(file_path+'/landing/pedidos.csv')
clientes = pd.read_csv(file_path+'/landing/clientes.csv')
vendedores = pd.read_csv(file_path+'/landing/vendedores.csv')
pagamentos = pd.read_csv(file_path+'/landing/pagamentos.csv')
dadospedidos = pd.read_csv(file_path+'/landing/dadospedidos.csv')

clientes.rename(columns = {'idgeralcliente':'idcliente'}, inplace = True)

pedidos = pd.merge(pedidos, dadospedidos, on='idpedido', how='left')
pedidos = pd.merge(pedidos, pagamentos, on='idpedido', how='left')
pedidos = pd.merge(pedidos, vendedores, on='idvendedor', how='left')
pedidos = pd.merge(pedidos, clientes, on='idcliente', how='left')

In [28]:
###DEMAIS TABELAS###
localizacoes = pd.read_csv(file_path+'/landing/geo.csv')
produtos = pd.read_csv(file_path+'/landing/dadosprodutos.csv')

###MERGE PRODUTOS E PEDIDOS###
pedidos = pd.merge(pedidos, produtos, left_on='idproduto', right_on='id_produto', how='left')

In [29]:
# Adicionar uma coluna de chave sequencial
df['sk_data'] = range(1, len(df) + 1)

# Adicionar uma coluna 'SK' como Surrogate Key
produtos['SK_Produto'] = produtos.index + 1
localizacoes['SK_Localizacoes'] = localizacoes.index + 1
pedidos['SK_Pedidos'] = pedidos.index + 1


In [30]:
# Salvando os dados de volta em um novo arquivo CSV com a SK adicionada
localizacoes.to_csv(file_path+'/bronze/localizacoes.csv')
produtos.to_csv(file_path+'/bronze/produtos.csv')
pedidos.to_csv(os.path.join(file_path,'bronze/pedidos.csv'), index=False)

# Salvar o DataFrame em um arquivo CSV no Google Drive
df.to_csv(os.path.join(file_path,'bronze\dim_data.csv'), index=False)